<a href="https://colab.research.google.com/github/FGC97/project1_Henry/blob/main/Project1_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import pandas as pd
import ast
import math
import numpy as np

# Importo los datasets

In [3]:
credits = pd.read_csv('/content/drive/MyDrive/Proyectos Henry/ML_Ops_Project/credits.csv', sep = ',')
movies = pd.read_csv('/content/drive/MyDrive/Proyectos Henry/ML_Ops_Project/movies_dataset.csv', sep = ',')

<ipython-input-3-5edfffbad0a3>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('/content/drive/MyDrive/Proyectos Henry/ML_Ops_Project/movies_dataset.csv', sep = ',')


In [ ]:
print(credits.info())
print(movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         

# **Ingesting Nested JSONs**

# DF credits

In [ ]:
# Los valores de cada elemento de "crew" y "cast" son una lista de diccionarios en formato de cadena de texto

print(type(credits['crew'][0]))
print(type(credits['cast'][0]))



<class 'str'>
<class 'str'>


## Crew: extraemos directores

In [ ]:
credits['crew'][1]

"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'department': 'Production', 'gender': 2, 'id': 511, 'job': 'Executive Producer', 'name': 'Larry J. Franco', 'profile_path': None}, {'credit_id': '52fe44bfc3a36847f80a7c89', 'department': 'Writing', 'gender': 2, 'id': 876, 'job': 'Screenplay', 'name': 'Jonathan Hensleigh', 'profile_path': '/l1c4UFD3g0HVWj5f0CxXAvMAGiT.jpg'}, {'credit_id': '52fe44bfc3a36847f80a7cdd', 'department': 'Sound', 'gender': 2, 'id': 1729, 'job': 'Original Music Composer', 'name': 'James Horner', 'profile_path': '/oLOtXxXsYk8X4qq0ud4xVypXudi.jpg'}, {'credit_id': '52fe44bfc3a36847f80a7c7d', 'department': 'Directing', 'gender': 2, 'id': 4945, 'job': 'Director', 'name': 'Joe Johnston', 'profile_path': '/fok4jaO62v5IP6hkpaaAcXuw2H.jpg'}, {'credit_id': '52fe44bfc3a36847f80a7cd7', 'department': 'Editing', 'gender': 2, 'id': 4951, 'job': 'Editor', 'name': 'Robert Dalva', 'profile_path': None}, {'credit_id': '573523bec3a368025100062c', 'department': 'Production', 'gender': 0, '

In [ ]:
# Uso la libreria ast para convertir crew en una lista:
credits['crew'] = credits['crew'].apply(ast.literal_eval)

In [ ]:
# Creo una función que recorra cada diccionario de la lista "crew" y me devuelva una lista con los directores de cada película:

def get_directors(crew):
    directors=[]
    for member in crew:
        if member["job"]=="Director":
            directors.append(member["name"])
    return directors

# Creo una nueva columna en el df "credits" aplicando la función "get_directors" a cada registro de la columna "crew":
credits['director']= credits['crew'].apply(get_directors)


In [ ]:
credits.head(3)

,cast,crew,id,director
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,[John Lasseter]
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,[Joe Johnston]
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,[Howard Deutch]


## Cast: Extraemos Actores

In [ ]:
credits['cast'][1]

"[{'cast_id': 1, 'character': 'Alan Parrish', 'credit_id': '52fe44bfc3a36847f80a7c73', 'gender': 2, 'id': 2157, 'name': 'Robin Williams', 'order': 0, 'profile_path': '/sojtJyIV3lkUeThD7A2oHNm8183.jpg'}, {'cast_id': 8, 'character': 'Samuel Alan Parrish / Van Pelt', 'credit_id': '52fe44bfc3a36847f80a7c99', 'gender': 2, 'id': 8537, 'name': 'Jonathan Hyde', 'order': 1, 'profile_path': '/7il5D76vx6QVRVlpVvBPEC40MBi.jpg'}, {'cast_id': 2, 'character': 'Judy Sheperd', 'credit_id': '52fe44bfc3a36847f80a7c77', 'gender': 1, 'id': 205, 'name': 'Kirsten Dunst', 'order': 2, 'profile_path': '/wBXvh6PJd0IUVNpvatPC1kzuHtm.jpg'}, {'cast_id': 24, 'character': 'Peter Shepherd', 'credit_id': '52fe44c0c3a36847f80a7ce7', 'gender': 0, 'id': 145151, 'name': 'Bradley Pierce', 'order': 3, 'profile_path': '/j6iW0vVA23GQniAPSYI6mi4hiEW.jpg'}, {'cast_id': 10, 'character': 'Sarah Whittle', 'credit_id': '52fe44bfc3a36847f80a7c9d', 'gender': 1, 'id': 5149, 'name': 'Bonnie Hunt', 'order': 4, 'profile_path': '/7spiVQwmr

In [ ]:
# Uso la libreria ast para convertir cast en una lista:
credits['cast'] = credits['cast'].apply(ast.literal_eval)

In [ ]:
# Creo una función que recorra cada diccionario de la lista "cast" y me devuelva una lista con los actores de cada película, pensado para el modelo de ML.

def get_actors(cast):
    actor_names = []
    for actor in cast:
        if 'name' in actor:
            actor_names.append(actor['name'])
    return actor_names


# Creo una nueva columna en el df "credits" aplicando la función "get_directors" a cada registro de la columna "crew":
credits['actors']= credits['cast'].apply(get_actors)

In [ ]:
credits.head()

,cast,crew,id,director,actors
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,[John Lasseter],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,[Joe Johnston],"[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,[Howard Deutch],"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop..."
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,[Forest Whitaker],"[Whitney Houston, Angela Bassett, Loretta Devi..."
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,[Charles Shyer],"[Steve Martin, Diane Keaton, Martin Short, Kim..."


## Dejo solo las columnas que me interesan

In [ ]:
credits = credits.drop(['cast', 'crew'], axis=1)

In [ ]:
credits

,id,director,actors
0,862,[John Lasseter],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,8844,[Joe Johnston],"[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,15602,[Howard Deutch],"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop..."
3,31357,[Forest Whitaker],"[Whitney Houston, Angela Bassett, Loretta Devi..."
4,11862,[Charles Shyer],"[Steve Martin, Diane Keaton, Martin Short, Kim..."
...,...,...,...
45471,439050,[Hamid Nematollah],"[Leila Hatami, Kourosh Tahami, Elham Korda]"
45472,111109,[Lav Diaz],"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe..."
45473,67758,[Mark L. Lester],"[Erika Eleniak, Adam Baldwin, Julie du Page, J..."
45474,227506,[Yakov Protazanov],"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa..."


In [ ]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        45476 non-null  int64 
 1   director  45476 non-null  object
 2   actors    45476 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [ ]:
# Verifico que no haya nulos
credits.isnull().sum()

id          0
director    0
actors      0
dtype: int64

# DF Movies

In [ ]:
# Función para pasar de string a lista, se controlan nulos.
def convert_to_dict(value):
    if pd.notnull(value):
        try:
            return ast.literal_eval(value)
        except (ValueError, SyntaxError):
            return None
    else:
        return None

## Columna **belongs_to_collection**

In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [ ]:
print(movies['belongs_to_collection'][0])
type(movies['belongs_to_collection'][0])

{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}


str

In [ ]:
movies['belongs_to_collection'].head(5)

0    {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                  NaN
2    {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                  NaN
4    {'id': 96871, 'name': 'Father of the Bride Col...
Name: belongs_to_collection, dtype: object

In [ ]:
# Aplicamos la función para pasarlo de str a lista
movies['belongs_to_collection'] = movies['belongs_to_collection'].apply(convert_to_dict)

In [ ]:
# Verificamos
print(movies['belongs_to_collection'][0])
type(movies['belongs_to_collection'][0])

{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}


dict

In [ ]:
# Quiero extraer los nombres de las colecciones pero falla por 3 datos tipo Float

# collection = movies['belongs_to_collection'].apply(lambda x: x['name'] if x is not None else None)
# collection

In [ ]:
# Compruebo donde estan esos valores float

for index, row in movies.iterrows():
    if isinstance(row["belongs_to_collection"], float):
        print(index, row["belongs_to_collection"])



19730 0.065736
29503 1.931659
35587 2.185485


In [ ]:
# Veo cuantos valores tiene la columna
print(len(movies['belongs_to_collection']))

# Veo cuantos valores tiene el dataframe
print(len(movies))
# 45466

45466
45466


In [ ]:
# Elimino esas 3 filas de todo el dataframe ya que tenian valores cambiados
movies = movies.drop(movies[movies['belongs_to_collection'].apply(lambda x: isinstance(x, float))].index)

In [ ]:
# Veo cuantos valores tiene el dataframe despues de borrar
len(movies)
#45563

45463

In [ ]:
# Definir la función para extraer el nombre de las franquicias
# Esta función asume que los valores float ya han sido eliminados previamente, por lo que no es necesario verificar si el valor es float o nan.
def extract_name(collection):
    if isinstance(collection, dict):
        return collection.get('name')
    return None

# def extract_name(collection):
#     if isinstance(collection, dict):
#         return collection['name']
#     elif isinstance(collection, float) and math.isnan(collection):
#         return None
#     else:
#         return None

# Aplicar la función a la columna para extraer los nombres y reemplazarla
movies['belongs_to_collection'] = movies['belongs_to_collection'].apply(extract_name)

print(movies['belongs_to_collection'])


0                  Toy Story Collection
1                                  None
2             Grumpy Old Men Collection
3                                  None
4        Father of the Bride Collection
                      ...              
45461                              None
45462                              None
45463                              None
45464                              None
45465                              None
Name: belongs_to_collection, Length: 45463, dtype: object


In [ ]:
# verificar los registros no nulos que tengo
len(movies['belongs_to_collection']) - movies['belongs_to_collection'].isnull().sum()

4491

## Columna **genres**

In [ ]:
movies['genres'].isnull().sum()

0

In [ ]:
print(movies['genres'][0])
type(movies['genres'][0])

[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]


str

In [ ]:
# Uso la función creada para pasarlo a lista de diccionarios
movies['genres'] = movies['genres'].apply(convert_to_dict)

In [ ]:
# Creo una función que recorra cada diccionario de la lista "genres" y me devuelva una lista con los generos de cada película, pensado para el modelo de ML.

def get_genres(movie_genres):
    genres_list = []
    for genre in movie_genres:
        genres_list.append(genre['name'])
    return genres_list

# Creo una nueva columna en el df "credits" aplicando la función "get_directors" a cada registro de la columna "crew":
movies['genres']= movies['genres'].apply(get_genres)

In [ ]:
movies.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,Toy Story Collection,30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,None,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,Grumpy Old Men Collection,0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


## Columna **production_companies**

In [ ]:
movies['production_companies'].isnull().sum()

3

In [ ]:
print(movies['production_companies'][0])
type(movies['production_companies'][0])

[{'name': 'Pixar Animation Studios', 'id': 3}]


str

In [ ]:
# Applying the convert_to_dict function to the 'production_companies' column
movies['production_companies'] = movies['production_companies'].apply(convert_to_dict)

In [ ]:
movies['production_companies'][0]


[{'name': 'Pixar Animation Studios', 'id': 3}]

In [ ]:
def get_companies(compania):
  companies=[]
  if compania != None and type(compania) != bool:
    try:
      for dicc in compania:
        companies.append(dicc["name"])
      return companies
    except (ValueError, SyntaxError):
      return None
  else:
    return None

movies['production_companies'] = movies['production_companies'].apply(get_companies)

In [ ]:
print(movies['production_companies'])
print(len(movies['production_companies']))

0                                [Pixar Animation Studios]
1        [TriStar Pictures, Teitler Film, Interscope Co...
2                           [Warner Bros., Lancaster Gate]
3                 [Twentieth Century Fox Film Corporation]
4             [Sandollar Productions, Touchstone Pictures]
                               ...                        
45461                                                   []
45462                                        [Sine Olivia]
45463                            [American World Pictures]
45464                                          [Yermoliev]
45465                                                   []
Name: production_companies, Length: 45463, dtype: object
45463


In [ ]:
movies['production_companies'].isnull().sum()
# Tengo 3 nulos

3

In [ ]:
# Contar las listas vacías en el Series, considerando valores None
empty_lists_count = movies['production_companies'].apply(lambda x: len(x) == 0 if x is not None else False).sum()

# Imprimir el resultado
print(empty_lists_count)

#REVISAR OTRA VEZ


11875


In [ ]:
# # Funcion que verifica listas vacias
# empty_count = df_movies["production_companies"].apply(lambda x: len(x) == 0).sum()
# empty_count

# #REVISAR OTRA VEZ

In [ ]:
null_indices = movies[movies['production_companies'].isnull()].index
print(null_indices)

Int64Index([19729, 29502, 35586], dtype='int64')


## Columna **production_countries**

In [ ]:
movies['production_countries'].isnull().sum()

3

In [ ]:
print(movies['production_countries'][0])
type(movies['production_countries'][0])

[{'iso_3166_1': 'US', 'name': 'United States of America'}]


str

In [ ]:
movies['production_countries'] = movies['production_countries'].apply(convert_to_dict)

In [ ]:
movies['production_countries'].head()

0    [{'iso_3166_1': 'US', 'name': 'United States o...
1    [{'iso_3166_1': 'US', 'name': 'United States o...
2    [{'iso_3166_1': 'US', 'name': 'United States o...
3    [{'iso_3166_1': 'US', 'name': 'United States o...
4    [{'iso_3166_1': 'US', 'name': 'United States o...
Name: production_countries, dtype: object

In [ ]:
# Ya está en lista. Aplico una función para obtener el nombre de los países:
def get_countries(countrie):
  countries=[]
  if countrie!= None and type(countrie)!=float:
    try:
      for dicc in countrie:
        countries.append(dicc["name"])
      return countries
    except (ValueError, SyntaxError):
      return None
  else:
      return None


# Modifico la columna genres para que solo tenga una lista con los países:
movies["production_countries"]=movies["production_countries"].apply(get_countries)

In [ ]:
# Función para extraer el valor de 'name' en cada fila
# Me devuelve None en las listas vacias por eso la diferencia de 6207 con olga

# def get_countries(row):
#     if isinstance(row, list) and len(row) > 0:
#         return row[0].get('name')
#     else:
#         return None

# # Aplicar la función al Series
# movies['production_countries'] = movies['production_countries'].apply(get_countries)



In [ ]:
# print(movies['production_countries'].isnull().sum())

# len(movies['production_countries'])

In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45463 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45463 non-null  object 
 1   belongs_to_collection  4491 non-null   object 
 2   budget                 45463 non-null  object 
 3   genres                 45463 non-null  object 
 4   homepage               7779 non-null   object 
 5   id                     45463 non-null  object 
 6   imdb_id                45446 non-null  object 
 7   original_language      45452 non-null  object 
 8   original_title         45463 non-null  object 
 9   overview               44509 non-null  object 
 10  popularity             45460 non-null  object 
 11  poster_path            45077 non-null  object 
 12  production_companies   45460 non-null  object 
 13  production_countries   45460 non-null  object 
 14  release_date           45376 non-null  object 
 15  re

In [ ]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        45476 non-null  int64 
 1   director  45476 non-null  object
 2   actors    45476 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [ ]:
movies['id'].duplicated().sum()

30

In [ ]:
credits['id'].duplicated().sum()

44

# Merge de DataFrames, credits & movies

In [ ]:
# Eliminar duplicados de movies y credits antes del merge
movies.drop_duplicates(subset='id', inplace=True)
credits.drop_duplicates(subset='id', inplace=True)

# 44 y 30


In [ ]:
# Paso a str
credits['id'] = credits['id'].astype(str)
movies['id'] =  movies['id'].astype(str)

In [ ]:
df_movies = pd.merge(movies, credits, on='id')

In [ ]:
df_movies.head(4)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,director,actors
0,False,Toy Story Collection,30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,[John Lasseter],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,False,None,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,[Joe Johnston],"[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,False,Grumpy Old Men Collection,0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,[Howard Deutch],"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop..."
3,False,None,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,[Forest Whitaker],"[Whitney Houston, Angela Bassett, Loretta Devi..."


In [ ]:
# Elimino columnas que no usaré
columns_to_drop = ['video','imdb_id','adult','original_title', 'poster_path', 'homepage']
df_movies = df_movies.drop(columns_to_drop, axis = 1)

In [ ]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45432 entries, 0 to 45431
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4488 non-null   object 
 1   budget                 45432 non-null  object 
 2   genres                 45432 non-null  object 
 3   id                     45432 non-null  object 
 4   original_language      45421 non-null  object 
 5   overview               44478 non-null  object 
 6   popularity             45429 non-null  object 
 7   production_companies   45429 non-null  object 
 8   production_countries   45429 non-null  object 
 9   release_date           45345 non-null  object 
 10  revenue                45429 non-null  float64
 11  runtime                45172 non-null  float64
 12  spoken_languages       45429 non-null  object 
 13  status                 45348 non-null  object 
 14  tagline                20401 non-null  object 
 15  ti

In [ ]:
df_movies['id'].duplicated().sum()

0

verificar datos por separado

In [ ]:
print(movies['id'].duplicated().sum())
print(type(movies['id'][0]))

0
<class 'str'>


In [ ]:
print(credits['id'].duplicated().sum())
print(type(credits['id'][0]))

0
<class 'str'>


In [ ]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45432 entries, 0 to 45431
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4488 non-null   object 
 1   budget                 45432 non-null  object 
 2   genres                 45432 non-null  object 
 3   id                     45432 non-null  object 
 4   original_language      45421 non-null  object 
 5   overview               44478 non-null  object 
 6   popularity             45429 non-null  object 
 7   production_companies   45429 non-null  object 
 8   production_countries   45429 non-null  object 
 9   release_date           45345 non-null  object 
 10  revenue                45429 non-null  float64
 11  runtime                45172 non-null  float64
 12  spoken_languages       45429 non-null  object 
 13  status                 45348 non-null  object 
 14  tagline                20401 non-null  object 
 15  ti

# **ETL**  

* Algunos campos, como `belongs_to_collection`, `production_companies` y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

* Los valores nulos de los campos `revenue`, `budget` deben ser rellenados por el número 0.

* Los valores nulos del campo `release date` deben eliminarse.

* De haber `fechas`, deberán tener el formato AAAA-mm-dd, además **deberán crear** la columna `release_year` donde extraerán el año de la fecha de estreno.

* Crear la columna con el retorno de inversión, llamada `return` con los campos `revenue` y `budget`, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

* Eliminar las columnas que no serán utilizadas, `video,imdb_id,adult,original_title,poster_path y homepage.`

## Revenue

In [ ]:
# Buscar valores nulos o vacíos para revenue
null_values = df_movies['revenue'].isnull()
empty_values = df_movies['revenue'] == ''

# Imprimir los resultados
print("Valores nulos en revenue:")
print(null_values.sum())

print("Valores vacíos en revenue:")
print(empty_values.sum())




Valores nulos en revenue:
3
Valores vacíos en revenue:
0


In [ ]:
# Reemplazar los valores nulos por 0
df_movies['revenue'] = df_movies['revenue'].fillna(0)

## Budget

In [ ]:
# Paso a float
df_movies['budget'] = df_movies['budget'].astype(float)

In [ ]:
# Buscar valores nulos o vacíos para budget
null_values = df_movies['budget'].isnull()
empty_values = df_movies['budget'] == ''

# Imprimir los resultados
print("Valores nulos en budget:")
print(null_values.sum())

print("Valores vacíos en budget:")
print(empty_values.sum())

Valores nulos en budget:
0
Valores vacíos en budget:
0


In [ ]:
# Reemplazar los valores nulos por 0
df_movies['budget'] = df_movies['budget'].fillna(0)

## Release_date

In [ ]:
df_movies['release_date'].isnull().sum()

87

In [ ]:
# Eliminar los valores nulos de la columna "release_date"
df_movies.dropna(subset=['release_date'], inplace=True)
#87


In [ ]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45345 entries, 0 to 45431
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4485 non-null   object 
 1   budget                 45345 non-null  float64
 2   genres                 45345 non-null  object 
 3   id                     45345 non-null  object 
 4   original_language      45334 non-null  object 
 5   overview               44404 non-null  object 
 6   popularity             45345 non-null  object 
 7   production_companies   45345 non-null  object 
 8   production_countries   45345 non-null  object 
 9   release_date           45345 non-null  object 
 10  revenue                45345 non-null  float64
 11  runtime                45099 non-null  float64
 12  spoken_languages       45345 non-null  object 
 13  status                 45265 non-null  object 
 14  tagline                20387 non-null  object 
 15  ti

In [ ]:
# Convertir el DataSeries a formato de fecha
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], format='%Y-%m-%d', errors='coerce') #errors='coerce': This parameter indicates how to handle errors during the conversion.

In [ ]:
# Crear nueva columna del año
df_movies["release_year"]= df_movies["release_date"].dt.year

In [ ]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45345 entries, 0 to 45431
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4485 non-null   object        
 1   budget                 45345 non-null  float64       
 2   genres                 45345 non-null  object        
 3   id                     45345 non-null  object        
 4   original_language      45334 non-null  object        
 5   overview               44404 non-null  object        
 6   popularity             45345 non-null  object        
 7   production_companies   45345 non-null  object        
 8   production_countries   45345 non-null  object        
 9   release_date           45345 non-null  datetime64[ns]
 10  revenue                45345 non-null  float64       
 11  runtime                45099 non-null  float64       
 12  spoken_languages       45345 non-null  object        
 13  s

In [ ]:
df_movies['revenue'].head()

0    373554033.0
1    262797249.0
2            0.0
3     81452156.0
4     76578911.0
Name: revenue, dtype: float64

## Return

Create the column with the return on investment, called return with the revenue and budget fields, dividing these last two revenue / budget, when there is no data available to calculate it, it must take the value 0.

In [ ]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45345 entries, 0 to 45431
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4485 non-null   object        
 1   budget                 45345 non-null  float64       
 2   genres                 45345 non-null  object        
 3   id                     45345 non-null  object        
 4   original_language      45334 non-null  object        
 5   overview               44404 non-null  object        
 6   popularity             45345 non-null  object        
 7   production_companies   45345 non-null  object        
 8   production_countries   45345 non-null  object        
 9   release_date           45345 non-null  datetime64[ns]
 10  revenue                45345 non-null  float64       
 11  runtime                45099 non-null  float64       
 12  spoken_languages       45345 non-null  object        
 13  s

In [ ]:
# Calcular el retorno de la inversión y asignar 0 cuando budget es 0

#df_movies['return'] = np.where(df_movies['budget'] == 0, 0, df_movies['revenue'] / df_movies['budget'])
df_movies['return'] = df_movies.apply(lambda row: row['revenue'] / row['budget'] if row['revenue'] and row['budget'] else 0, axis=1)
df_movies.head()


,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,director,actors,release_year,return
0,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,[John Lasseter],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",1995,12.451801
1,None,65000000.0,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,[Joe Johnston],"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",1995,4.043035
2,Grumpy Old Men Collection,0.0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,[Howard Deutch],"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",1995,0.000000
3,None,16000000.0,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,[Forest Whitaker],"[Whitney Houston, Angela Bassett, Loretta Devi...",1995,5.090760
4,Father of the Bride Collection,0.0,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.387519,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,[Charles Shyer],"[Steve Martin, Diane Keaton, Martin Short, Kim...",1995,0.000000


In [ ]:
df_movies['return'].isnull().sum()

0

In [ ]:
df_movies['return'][0]

12.4518011

In [ ]:
df_movies['return'].head(10)

0    12.451801
1     4.043035
2     0.000000
3     5.090760
4     0.000000
5     3.123947
6     0.000000
7     0.000000
8     1.838576
9     6.072311
Name: return, dtype: float64

# Rearrange columns order

In [ ]:
columnas = ['id', 'title', 'belongs_to_collection', 'genres',  'budget', 'original_language', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'vote_average', 'vote_count', 'director', 'actors', 'release_year', 'return']


In [ ]:
df_movies = df_movies[columnas]

# Exporting CSV

In [ ]:
savefile_path = '/content/drive/MyDrive/Proyectos Henry/ML_Ops_Project/df_movies.csv'
df_movies.to_csv(savefile_path, index=False)

In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45433 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45433 non-null  object 
 1   belongs_to_collection  4488 non-null   object 
 2   budget                 45433 non-null  object 
 3   genres                 45433 non-null  object 
 4   homepage               7774 non-null   object 
 5   id                     45433 non-null  object 
 6   imdb_id                45416 non-null  object 
 7   original_language      45422 non-null  object 
 8   original_title         45433 non-null  object 
 9   overview               44479 non-null  object 
 10  popularity             45430 non-null  object 
 11  poster_path            45047 non-null  object 
 12  production_companies   45430 non-null  object 
 13  production_countries   45430 non-null  object 
 14  release_date           45346 non-null  object 
 15  re